In [3]:
# Allow auto-reload of external modules 
%reload_ext autoreload
%autoreload 2

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import scipy as scp
import scipy.misc

import numpy as np
import tensorflow as tf
import nn
from network.fcn_instance import InstanceFCN8s
import data_utils as dt

In [39]:
from PIL import Image
def load_image(fname):
    """
    Load input image and preprocess for using pretrained weight from Caffee:
    - cast to float
    - switch channels RGB -> BGR
    - subtract mean
    - transpose to channel x height x width order
    """
    #print('Loading img:%s'%fname)
    try:
        img = Image.open(fname)
    except IOError as e:
        print('Warning: no image with name %s!!'%fname)

    image = np.array(img, dtype=np.float32)
    #image = image[:,:,::-1]     # RGB -> BGR
    return image

fd = '10000_20'
for k in range(5):
    x = load_image('data/test_city_instance/%s/person_%d.png'%(fd,k))
    (x0,x1) = np.where(x > 0)
    pid = []
    for i in range(x0.shape[0]):
        id = x[x0[i],x1[i]]
        if id not in pid:
            pid.append(id)

    x = load_image('data/test_city_instance/%s/car_%d.png'%(fd,k))
    (x0,x1) = np.where(x > 0)
    cid = []
    for i in range(x0.shape[0]):
        id = x[x0[i],x1[i]]
        if id not in cid:
            cid.append(id)

    print(pid, cid)

[1.0, 2.0] [1.0, 2.0, 4.0, 3.0, 6.0]
[1.0, 2.0] [1.0, 2.0, 3.0, 4.0, 6.0]
[1.0, 2.0] [1.0, 2.0, 4.0, 3.0, 6.0]
[1.0, 2.0] [1.0, 2.0, 4.0, 3.0, 5.0, 6.0]
[1.0, 2.0] [1.0, 2.0, 4.0, 3.0, 6.0]


In [20]:
(x0,x1) = np.where(x > 0)
ids = []
for i in range(x0.shape[0]):
    id = x[x0[i],x1[i]]
    if id not in ids:
        ids.append(id)

print(ids)

[1.0, 4.0, 5.0, 3.0, 2.0, 6.0, 7.0, 15.0]


In [7]:
'''
Testing script for fcn32 without skip architecture.
'''
from scipy.misc import toimage
from scipy.misc import imsave

def get_gtmask_tuple(gt_sparse, inst_id):
    '''
    gt_sparse: [1, H, W, 1] sparse gt mask
    inst_id: 1-30 (MUST not start from 0!)
    Return: tuple (0/1 mask, #1s)
    '''
    gt_shape = tf.shape(gt_sparse)
    gt_sparse = tf.reshape(gt_sparse, [gt_shape[1], gt_shape[2], 1])
    where = tf.equal(gt_sparse, inst_id)
    indices = tf.where(where)
    val = tf.ones((tf.shape(indices)[0],),tf.float32)
    mask = tf.sparse_to_dense(indices,[1024, 2048,1],val)
    mask = tf.reshape(mask, [1, 1024, 2048, 1])

    return (mask, tf.shape(indices)[0])


# Import training and validation dataset
test_data_config = {'city_dir':"../data/CityDatabase",
                     'randomize': False,
                     'use_gt_mask': True,
                     'seed': None,
                     'dataset':'train'
                     }
params = {'num_classes': 20, 'max_instance': 20, 
          'target_class':{11:'person', 13:'car'},
          'trained_weight_path':'../data/val_weights/city_instance_100.npy'}

test_dataset = dt.CityDataSet(test_data_config)
iterations = 1


with tf.Session() as sess:
    # Initialization
    image = tf.placeholder(tf.float32, shape=[1, None, None, 3])
    train_gt_mask = tf.placeholder(tf.int32, shape=[1, None, None, len(params['target_class'])])
    gt_mask_list = tf.split(3, 2, train_gt_mask)
    total_pixel = 1024 * 2048
    gt_list = []

    for i in range(2):
        gt = gt_mask_list[i]
        for j in range(params['max_instance']):
            (gt_j, inst_pixel) = get_gtmask_tuple(gt, j+1)
            weight = tf.cond(tf.equal(inst_pixel, 0), lambda: tf.constant(1, dtype=tf.float64) , lambda: (total_pixel - inst_pixel) / inst_pixel )
            weight = tf.cast(weight, tf.float32)
            gt_j = tf.cast(gt_j, tf.float32)
            gt_list.append(gt_j)
    
    init = tf.initialize_all_variables()
    sess.run(init)

    for i in range(iterations):
        # Load data, Already converted to BGR
        next_pair = test_dataset.next_batch()
        next_pair_image = next_pair[0]
        train_feed_dict = {train_img: next_pair_image,
                           train_gt_mask: next_pair_gt_mask,}

        gt_list_ = sess.run(gt_list, feed_dict=train_feed_dict)
#         pname = '../data/test_city_instance/person_%d.png'%i
#         cname = '../data/test_city_instance/car_%d.png'%i
#         toimage(predict_[0], high=params['max_instance'], low=0, cmin=0, cmax=params['max_instance']).save(pname)
#         toimage(predict_[1], high=params['max_instance'], low=0, cmin=0, cmax=params['max_instance']).save(cname)
        
        #imsave('../data/test_city_instance/person_%d.png'%i,predict_[0])
        #imsave('../data/test_city_instance/car_%d.png'%i, predict_[1])


Load train dataset
Loading masks
Training images:0 Ground Truth images:0
Running the inference ...


IndexError: list index out of range

In [106]:
x = np.random.rand(1,4,4,3)
w = tf.Variable(x)
k = tf.Variable(np.random.rand(3,3,2,5))
lis = [0,2]
lis_ = []
for i in lis:
    t = tf.slice(w, [0,0, 0, i], [1,4, 4, 1])
    lis_.append(t)
new = tf.concat(3,lis_)
conv = tf.nn.depthwise_conv2d(new, k,
                              strides=[1, 1, 1, 1],
                              padding='SAME')


init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    m = sess.run(new)
    c = sess.run(conv)

In [107]:
print(m.shape)
print(c.shape)


(1, 4, 4, 2)
(1, 4, 4, 10)


In [94]:
data_dict = dt.load_weight('data/vgg16.npy')

Successfully load weight file from /Users/jennyzhou/GitLocalRepository/APC_InstanceSegmentation/core/data/vgg16.npy.


In [97]:
for x in data_dict:
    print(data_dict[x][0].shape, data_dict[x][1].shape)

(3, 3, 512, 512) (512,)
(25088, 4096) (4096,)
(3, 3, 512, 512) (512,)
(4096, 4096) (4096,)
(4096, 1000) (1000,)
(3, 3, 512, 512) (512,)
(3, 3, 256, 512) (512,)
(3, 3, 512, 512) (512,)
(3, 3, 512, 512) (512,)
(3, 3, 256, 256) (256,)
(3, 3, 256, 256) (256,)
(3, 3, 128, 256) (256,)
(3, 3, 3, 64) (64,)
(3, 3, 64, 64) (64,)
(3, 3, 128, 128) (128,)
(3, 3, 64, 128) (128,)
